In [44]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import time
import json

In [45]:
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
wd_data=requests.get(url)

In [46]:
soup=BeautifulSoup(wd_data.text,"html.parser")
print(soup.select('title'))

[<title>List of postal codes of Canada: M - Wikipedia</title>]


In [47]:
table=soup.find_all('table')[0]
tablerows=table.find_all('tr')

res=[]
for tr in tablerows:
    td=tr.find_all('td')
    row=[tr.text.strip() for tr in td]
    if row:
        res.append(row)
df=pd.DataFrame(res,columns=["Postcode","Borough","Neighbourhood"]) 
print(df.shape)
df.head(10)

(288, 3)


,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Not assigned
9,M8A,Not assigned,Not assigned


In [48]:
df = df.drop(df[df["Borough"].str.contains('Not assigned')].index)
df.reset_index(drop=True)
df.head(10)

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Not assigned
10,M9A,Etobicoke,Islington Avenue
11,M1B,Scarborough,Rouge
12,M1B,Scarborough,Malvern


In [49]:
dfg1=df['Postcode']
dfg1
dfg2=dfg1.unique()
dfg2
dfres=pd.DataFrame(columns=['Postcode','Borough','Neighbourhood'])

for pc in dfg2:
    rr=df[df['Postcode']==pc]
    if rr.shape[0]>1:
        tt=rr.iloc[0,2]
        for i in range(1,rr.shape[0]):
            tt=tt+','+rr.iloc[i,2]
        add={'Postcode':rr.iloc[0,0],'Borough':rr.iloc[0,1],'Neighbourhood':tt}
        dfres=dfres.append(add,ignore_index=True)        
    else:
        if rr.iloc[0,2]=='Not assigned':
            rr['Neighbourhood']=rr['Borough']
        add={'Postcode':rr.iloc[0,0],'Borough':rr.iloc[0,1],'Neighbourhood':rr.iloc[0,2]}
        dfres=dfres.append(add,ignore_index=True)    
       
print(dfres.shape)
dfres.head(10)

/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/ipykernel/__main__.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


(103, 3)


,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront,Regent Park"
3,M6A,North York,"Lawrence Heights,Lawrence Manor"
4,M7A,Queen's Park,Queen's Park
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Rouge,Malvern"
7,M3B,North York,Don Mills North
8,M4B,East York,"Woodbine Gardens,Parkview Hill"
9,M5B,Downtown Toronto,"Ryerson,Garden District"
